# Query cellxgene-census using TileDB-SOMA

The [first guide](cellxgene-census) queried metadata and h5ad files directly through LaminDB.

This guide uses the TileDB-SOMA API to run similar queries.

## Setup

Load your LaminDB instance for storing queried data:

In [ ]:
!lamin load test-census

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import cellxgene_census

census_version = "2023-07-25"

## Create lookup objects

We use metadata records in the `laminlabs/cellxgene-census` instance to generate lookups:

In [ ]:
db_handle = "laminlabs/cellxgene-census"

organisms = lb.Organism.lookup(return_field="scientific_name", using=db_handle)
modalities = ln.Modality.lookup(using=db_handle)
features = ln.Feature.lookup(return_field="name", using=db_handle)
assays = lb.ExperimentalFactor.lookup(return_field="name", using=db_handle)
cell_types = lb.CellType.lookup(return_field="name", using=db_handle)
tissues = lb.Tissue.lookup(return_field="name", using=db_handle)
ulabels = ln.ULabel.lookup(using=db_handle)
suspension_types = ulabels.is_suspension_type.children.all().lookup(return_field="name")

## Query data

In [ ]:
value_filter = (
    f'{features.tissue} == "{tissues.brain}" and {features.cell_type} in'
    f' ["{cell_types.microglial_cell}", "{cell_types.neuron}"] and'
    f' {features.suspension_type} == "{suspension_types.cell}" and {features.assay} =='
    f' "{assays.ln_10x_3_v3}"'
)

In [ ]:
value_filter

In [ ]:
%%time

with cellxgene_census.open_soma(census_version=census_version) as census:
    # Reads SOMADataFrame as a slice
    cell_metadata = census["census_data"][organisms.human].obs.read(
        value_filter=value_filter
    )

    # Concatenates results to pyarrow.Table
    cell_metadata = cell_metadata.concat()

    # Converts to pandas.DataFrame
    cell_metadata = cell_metadata.to_pandas()

In [ ]:
cell_metadata.shape

In [ ]:
cell_metadata.head()

## Create AnnData

In [ ]:
%%time

with cellxgene_census.open_soma(census_version=census_version) as census:
    adata = cellxgene_census.get_anndata(
        census=census,
        organism=organisms.human,
        obs_value_filter=value_filter,
        column_names={
            "obs": [
                features.assay,
                features.assay_ontology_term_id,
                features.cell_type,
                features.cell_type_ontology_term_id,
                features.tissue,
                features.tissue_ontology_term_id,
                features.disease,
                features.disease_ontology_term_id,
                features.suspension_type,
            ]
        },
    )

In [ ]:
adata.var = adata.var.set_index("feature_id")

In [ ]:
adata

In [ ]:
adata.var.head()

In [ ]:
adata.obs.head()

## Register the queried AnnData

In [ ]:
ln.track()

Register the `AnnData` object:

In [ ]:
lb.settings.organism = "human"
modalities = ln.Modality.lookup()
features = ln.Feature.lookup().dict()

In [ ]:
file = ln.File.from_anndata(
    adata,
    description=(
        "microglial and neuron cell data from 10x 3' v3 in brain queried from Census"
    ),
    field=lb.Gene.ensembl_gene_id,
    modality=modalities.rna,
)

In [ ]:
file.save()

Link validated metadata:

In [ ]:
for col in adata.obs.columns:
    if not col.endswith("ontology_term_id"):
        file.labels.add(adata.obs[col], features.get(col))

In [ ]:
file.describe()

In [ ]:
file.view_flow()

In [ ]:
# clean up test instance
!lamin delete --force test-census
!rm -r ./test-census